# Layers

The most general layers in `kgcnn` take normal and ragged tensor as input. The graph oriented operations are im


1. The most general layers that kept maintained beyond different models with proper documentation are located in `kgcnn.layers`. These are:
    * `kgcnn.layers.attention` Layers for graph attention.
    * `kgcnn.layers.casting` Layers for casting tensor formats.
    * `kgcnn.layers.conv` Basic convolution layers.
    * `kgcnn.layers.gather` Layers around tf.gather.
    * `kgcnn.layers.geom` Geometry operations.
    * `kgcnn.layers.message` Message passing base layer.
    * `kgcnn.layers.mlp` Multi-layer perceptron for graphs.
    * `kgcnn.layers.modules` Keras layers and modules to support ragged tensor input.
    * `kgcnn.layers.norm` Normalization layers for graph tensors.
    * `kgcnn.layers.pooling` General layers for standard aggregation and pooling.
    * `kgcnn.layers.relational` Relational message processing.
    * `kgcnn.layers.set2set` Set2Set type architectures for e.g. pooling nodes.
    * `kgcnn.layers.update` Some node/edge update layers.


> **NOTE**: Please check https://kgcnn.readthedocs.io/en/latest/kgcnn.layers.html for documentation of each layer.

## Implementaion details

Most tensorflow methods already support ragged tensors, which can be looked up here: https://www.tensorflow.org/api_docs/python/tf/ragged. 

For using keras layers, most layers in `kgcnn` inherit from `kgcnn.layers.base.GraphBaseLayer` which adds some utility methods and arguments, such as the `ragged_validate` parameter that is used for ragged tensor creation.

In [1]:
import tensorflow as tf
from kgcnn.layers.base import GraphBaseLayer

class NewLayer(GraphBaseLayer):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def call(self, inputs, **kwargs):
        # Do something in call.
        return inputs

new_layer = NewLayer(ragged_validate=False)
print(new_layer._supports_ragged_inputs)

True


Since ragged tensors can result in quite a performance loss due to shape checks of ragged dimensions on runtime, it is recommended to directly work with the values tensor/information (if possible) and to use `ragged_validate` to `False` in production. An example is the `tf.ragged.map_flat_values` method.

Utility methods of `GraphBaseLayer` to work with values directly are `assert_ragged_input_rank` and `map_values` . Note that this can speed up models and is essentially equal to a disjoint graph tensor representation. 
However, with ragged tensors there is also the possibility to try `tf.vectorized_map` or `tf.map_fn` if values can not be accessed.

Here is an example of how to use `assert_ragged_input_rank` and `map_values` . With `assert_ragged_input_rank` it can be ensured that a ragged tensor is given in `call` by casting padded+mask or normal tensor (for example in case of equal sized graphs) to a ragged version, in order to accesss e.g. `inputs.values` etc.
With `map_values` a function can be directly applied to the values tensor or a list of value tensors. Axis argument can refer to the ragged tensor shape but the `map_values` is restricted to ragged rank of one. Fallback for `map_values` is applying the function directly to its input.

In [2]:
class NewLayer(GraphBaseLayer):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def call_v1(self, inputs, **kwargs):
        inputs = self.assert_ragged_input_rank(inputs, ragged_rank=1)
        return tf.RaggedTensor.from_row_splits(
            tf.exp(inputs.values), inputs.row_splits, validate=self.ragged_validate)

    def call_v2(self, inputs, **kwargs):
        # Possible kwargs for function can be added. Can have axis argument (special case).
        return self.map_values(tf.exp, inputs)
    
new_layer = NewLayer()

In [3]:
x = tf.ones((2, 3, 4))
x_ragged = tf.ragged.constant([[[1.0, 1.0, 1.0, 1.0]],[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0]]], ragged_rank=1)

print("x (v1):", x.shape, new_layer.call_v1(x).shape)
print("x (v2):", x.shape, new_layer.call_v2(x).shape)
print("x_ragged (v1):", x_ragged.shape, new_layer.call_v1(x_ragged).shape)
print("x_ragged (v2):", x_ragged.shape, new_layer.call_v2(x_ragged).shape)

x (v1): (2, 3, 4) (2, None, 4)
x (v2): (2, 3, 4) (2, 3, 4)
x_ragged (v1): (2, None, 4) (2, None, 4)
x_ragged (v2): (2, None, 4) (2, None, 4)


> **NOTE**: You can find this page as jupyter notebook in https://github.com/aimat-lab/gcnn_keras/tree/master/docs/source